# Databrick Tutorial
### Creating a databricks workspace and connecting to azure storage
1. Azure: Go to the azure and search for databricks. Enter the details and create a databricks workspace
2. Azure: search for "Access Connector for databrick" in azure and create one
3. Azure: Create a storage account and two containers one for source and one destination
4. Azure: Go to storage account and give "storage blob data contributor" permission to the "access connector for databricks"
5. Databricks: Go to catalogs and create a new storage credential and provide "access connector ID" to the storage credentials
6. Databricks: create a new external location and provde the file path in the following format "abfss://<container_name>@<storage_account_name>.dfs.core.windows.net/<path>/file.csv. also select the storage credentials you created earlier
7. After the access the file from the notebook using the below command
    file_path = "abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv"
    file_content = dbutils.fs.head(file_path)
    print(file_content)


**Accessing a file in the Azure ADLS 2**

In [0]:
#Option 1
file_path = "abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv"
file_content = dbutils.fs.head(file_path)
print(file_content) 

#Option 2
# df = spark.read.option("header", "true").csv("abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv")
# df.display()

**Listing files in a contaioner**

In [0]:
display(dbutils.fs.ls("abfss://source@rxdbstorageaccount.dfs.core.windows.net/"))



**Copy file from one container to another**


In [0]:
df = spark.read.option("header", "true").csv("abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv")
df.write.mode("overwrite").csv("abfss://destination@rxdbstorageaccount.dfs.core.windows.net/Sales_Copy")


In [0]:
dbutils.fs.ls('/databricks-datasets/')

## Transformations

**Select and Drop**

In [0]:
# Read the file from the Azure storage into a spark data frame
df = spark.read.option("header", "true").csv("abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv")
#display the data in spark environment
#df.display()
df_select = df.select("Item_Identifier", "Item_Outlet_Sales", "Item_Weight")

#df_select.display()
# not formatted highly and is suppoprted in opyspark
#df_select.show()

# drop specific columns from the data frame
df_drop=df_select.drop("Item_Weight")
df_drop.display()



**withColumnRenamed, withColumn, lit, cast, when**

In [0]:
# df = spark.read.option("header", "True").csv("abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv")
# df = df.withColumnRenamed("Item_Identifier", "ItemID")
# df.display()

import pyspark.sql.functions as F
from pyspark.sql.functions import col, when, lit
df = spark.read.option("header", "True").csv("abfss://source@rxdbstorageaccount.dfs.core.windows.net/Sales.csv")
from pyspark.sql.types import FloatType
df = df.withColumn("Item_Weight_Category", when(col("Item_Weight").cast(FloatType())>10, lit("high")).otherwise(when (col("Item_Weight").cast(FloatType())>5, lit("medium")).otherwise(lit("low"))))

df.display()

